In [205]:
import numpy as np
import pandas as pd
import sqlite3 as sql
import csv

In [97]:
data1 = pd.read_csv('credit_card.csv')
data1.head()

,ID,Gender,Has a car,Has a property,Children count,Income,Employment status,Education level,Marital status,Dwelling,Age,Employment length,Has a mobile phone,Has a work phone,Has a phone,Has an email,Job title,Family member count,Account age,Is high risk
0,5037048,M,Y,Y,0,135000.0,Working,Secondary / secondary special,Married,With parents,-16271,-3111,1,0,0,0,Core staff,2.0,-17.0,0
1,5044630,F,Y,N,1,135000.0,Commercial associate,Higher education,Single / not married,House / apartment,-10130,-1651,1,0,0,0,Accountants,2.0,-1.0,0
2,5079079,F,N,Y,2,180000.0,Commercial associate,Secondary / secondary special,Married,House / apartment,-12821,-5657,1,0,0,0,Laborers,4.0,-38.0,0
3,5112872,F,Y,Y,0,360000.0,Commercial associate,Higher education,Single / not married,House / apartment,-20929,-2046,1,0,0,1,Managers,1.0,-11.0,0
4,5105858,F,N,N,0,270000.0,Working,Secondary / secondary special,Separated,House / apartment,-16207,-515,1,0,1,0,NaN,1.0,-41.0,0


In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
connection = sql.connect('example.db')

In [ ]:
cursor = connection.cursor()

In [ ]:
cursor.execute('''DROP TABLE data''')

In [ ]:
query = ''' 
CREATE TABLE IF NOT EXISTS data (
id INT,
gender VARCHAR(5),
has_a_car VARCHAR(5),
has_a_property VARCHAR(5),
children_count INT,
income FLOAT,
employment_status TEXT,
education_level TEXT,
marital_status TEXT,
dwelling TEXT,
age INT,
employment_length INT,
has_a_mobile_phone INT,
has_a_work_phone INT,
has_a_phone INT,
has_an_email INT,
job_title TEXT,
familty_member_count FLOAT,
account_age FLOAT,
is_high_risk INT) '''

cursor.execute(query)

In [ ]:
csv_file = 'credit_card.csv'
with open(csv_file, 'r', newline='') as file:
    csv_reader = csv.reader(file)
    next(csv_reader)  # Skip the header row if it exists
    for row in csv_reader:
        cursor.execute('INSERT INTO data VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)', row)  
connection.commit()

In [102]:
query = '''
SELECT * FROM data '''
cursor.execute(query)
data = cursor.fetchall()
for row in data:
    print(row)

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



(5024604, 'F', 'N', 'Y', 0, 630000.0, 'Pensioner', 'Secondary / secondary special', 'Single / not married', 'House / apartment', -21982, 365243, 1, 0, 1, 0, '', 1.0, -8.0, 0)
(5061729, 'F', 'N', 'Y', 0, 135000.0, 'Pensioner', 'Secondary / secondary special', 'Single / not married', 'House / apartment', -23929, 365243, 1, 0, 1, 0, '', 1.0, -37.0, 0)
(5045272, 'F', 'N', 'Y', 2, 180000.0, 'Working', 'Secondary / secondary special', 'Married', 'House / apartment', -13787, -3082, 1, 0, 0, 0, 'Core staff', 4.0, -26.0, 0)
(5042010, 'F', 'N', 'N', 0, 157500.0, 'Working', 'Incomplete higher', 'Married', 'House / apartment', -12402, -917, 1, 1, 0, 0, 'Managers', 2.0, -24.0, 0)
(5021981, 'M', 'N', 'Y', 0, 315000.0, 'Pensioner', 'Secondary / secondary special', 'Married', 'House / apartment', -20282, 365243, 1, 0, 0, 0, '', 2.0, -28.0, 0)
(5056056, 'M', 'Y', 'Y', 0, 135000.0, 'Working', 'Secondary / secondary special', 'Married', 'House / apartment', -18810, -3830, 1, 1, 0, 0, 'Laborers', 2.0, -34

In [130]:
#Let us see how much values we have in the dataset

query = ''' SELECT COUNT(*) AS total_count FROM data'''
data = pd.read_sql_query(query, connection)
print(data)

   total_count
0        29165


In [163]:
# Let us see how many males and how many females we have in the given dataset

query = '''
SELECT SUM(CASE WHEN gender = 'M' THEN 1 ELSE 0 END) AS male_count,
       SUM(CASE WHEN gender = 'F' THEN 1 ELSE 0 END) AS female_count
FROM data
'''
cursor.execute(query)
data = cursor.fetchall()
print('There are {} Males and {} Females in the dataset'.format(data[0][0], data[0][1]))

There are 9616 Males and 19549 Females in the dataset


So, we can conclude that number of females are approximately twice the number of males credit card buyers 

In [178]:
# Let us see count of  employment status  

query = '''
SELECT employment_status, 
SUM(CASE WHEN gender = 'M' THEN 1 ELSE 0 END) AS total_male_count,
SUM(CASE WHEN gender = 'F' THEN 1 ELSE 0 END) AS total_female_count
FROM data
GROUP BY employment_status
ORDER BY total_male_count DESC
'''
cursor.execute(query)
data = pd.read_sql_query(query, connection)
print(data)

      employment_status  total_male_count  total_female_count
0               Working              5895                9161
1  Commercial associate              2360                4441
2             Pensioner               723                4197
3         State servant               638                1743
4               Student                 0                   7


1. Working People and Commerical associate are top credit card buyers.
2. Meaning, both employee and commerical people have been showing intrest in buying credit card
3. Female pensioner are as well one of the strong buyers of credit card. 

In [161]:
# Let us see count of education level 

query = '''
SELECT education_level, 
SUM(CASE WHEN gender = 'M' THEN 1 ELSE 0 END)AS total_male_count,
SUM(CASE WHEN gender = 'F' THEN 1 ELSE 0 END)AS total_female_count
FROM data
GROUP BY education_level
ORDER BY total_male_count DESC
'''
data = pd.read_sql_query(query, connection)
print(data)

                 education_level  total_male_count  total_female_count
0  Secondary / secondary special              6535               13268
1               Higher education              2553                5357
2              Incomplete higher               412                 717
3                Lower secondary               109                 189
4                Academic degree                 7                  18


1. Majority of credit card buyers have completed Secondary education. 
2. There is negative correlation between the education grade and amount of credit card buyers, intresting. 

In [168]:
# Let us see count marital status

query = '''
SELECT marital_status, 
SUM(CASE WHEN gender = 'M' THEN 1 ELSE 0 END) AS total_male_count,
SUM(CASE WHEN gender = 'F' THEN 1 ELSE 0 END) AS total_female_count
FROM data
GROUP BY marital_status
ORDER BY total_male_count DESC
'''
data = pd.read_sql_query(query, connection)
print(data)

         marital_status  total_male_count  total_female_count
0               Married              7317               12727
1  Single / not married              1251                2613
2        Civil marriage               674                1638
3             Separated               351                1361
4                 Widow                23                1210


1. Married people are largest credit card buyers.
2. Widows are smallest credit card buyers
3. un-married single people are showing intrest in buying credit card. These people could be good target for marketing.

In [180]:
# Let us see children count

query = '''
SELECT children_count,
SUM(CASE WHEN gender = 'M' THEN 1 ELSE 0 END) AS total_male_count,
SUM(CASE WHEN gender = 'F' THEN 1 ELSE 0 END) AS total_female_count
FROM data 
GROUP BY  children_count
ORDER BY total_male_count DESC
'''
data = pd.read_sql_query(query, connection)
print(data)

   children_count  total_male_count  total_female_count
0               0              6137               14006
1               1              2223                3780
2               2              1110                1514
3               3               122                 201
4               4                15                  37
5               5                 7                   8
6              14                 2                   0
7              19                 0                   1
8               7                 0                   2


1. Newly married people i.e. people with no children are the largest credit card buyers.
2. There is exponential decrease in credit card buying as the number of children increases.
3. Male and female having at most three children provide good market for credit card companies. 

In [185]:
# Let us see the propety count of credit card holders

query = '''
SELECT gender,
SUM(CASE WHEN has_a_property = 'Y' THEN 1 ELSE 0 END) AS has_property,
SUM(CASE WHEN has_a_property = 'N' THEN 1 ELSE 0 END) AS no_property
FROM data
GROUP BY gender
'''
data = pd.read_sql_query(query, connection)
print(data)

  gender  has_property  no_property
0      F         13415         6134
1      M          6142         3474


1. People who have property are twice the amount of people who don't have property.
2. This means people living on rented apartment don't usually buy credit card.

In [188]:
# Let us see the count of credit card holders who have and don't has a car.

query = '''
SELECT gender,
SUM(CASE WHEN has_a_car = 'Y' THEN 1 ELSE 0 END) AS has_a_car,
SUM(CASE WHEN has_a_car = 'N' THEN 1 ELSE 0 END) AS no_car
FROM data
GROUP BY gender
'''
data = pd.read_sql_query(query, connection)
print(data)

  gender  has_a_car  no_car
0      F       5003   14546
1      M       6034    3582


1. Females who don't has a car are biggest credit card holders.
2. Males who has a car are biggest credit card holders.

In [202]:
# Let us see employment status of people with or without car who hold credit card

query = '''
SELECT employment_status, has_a_car,
SUM(CASE WHEN gender = 'M' THEN 1 ELSE 0 END) AS total_male_count,
SUM(CASE WHEN gender = 'F' THEN 1 ELSE 0 END) AS total_female_count
FROM data
GROUP BY employment_status, has_a_car
ORDER BY total_male_count DESC, total_female_count DESC
'''
data = pd.read_sql_query(query, connection) 
print(data)

      employment_status has_a_car  total_male_count  total_female_count
0               Working         Y              3730                2547
1               Working         N              2165                6614
2  Commercial associate         Y              1478                1406
3  Commercial associate         N               882                3035
4             Pensioner         Y               418                 628
5         State servant         Y               408                 422
6             Pensioner         N               305                3569
7         State servant         N               230                1321
8               Student         N                 0                   7


In [191]:
# Let us see count of each Dwelling category

query = '''
SELECT dwelling, 
SUM(CASE WHEN gender = 'M' THEN 1 ELSE 0 END) AS total_male_count,
SUM(CASE WHEN gender = 'F' THEN 1 ELSE 0 END) AS total_female_count
FROM data
GROUP BY dwelling
ORDER BY total_male_count DESC 
'''
data = pd.read_sql_query(query, connection)
print(data)

              dwelling  total_male_count  total_female_count
0    House / apartment              8331               17728
1         With parents               610                 796
2  Municipal apartment               273                 639
3     Rented apartment               227                 226
4     Office apartment               116                  92
5      Co-op apartment                59                  68


1. People living in houses and apartment are biggest credit card buyers.
2. People living in co-op apartment has lowers credit card buyers.
3. Makes living with parents are showing intrest in buying credit card as well.

# Conclusion:

1. Married women with a maximum of three children, who possess property but do not possess a car, are considered as credit card consumers with a high probability. 
2.  Married men with a maximum of three children, who possess property and a car, are also a target audience with a high probability for credit card sales. 
3. In terms of marketing, single men and women, retired women and men living with their parents, are among the most favorable targets. 
4. On the other hand, students, individuals residing in cooperative apartments, and married individuals with more than four children are the least likely to be consumers.
5. Also, education grade and credit card buyers count has negative correlation.